# Project 4: Predicting West Nile Virus
## Team Purple Cobras - General Assembly Boston - DSI
## September 2018

# Feature Engineering Notebook

Goal: Build a classification model to predict where and when West Nile Virus will be found in Chicago.

The purpose of this notebook is to build features from the data, explore some relationships and try to identify which features will be most useful, then start building some models. This notebook will also include some EDA and model-building, although I expect to have separate notebooks handling the bulk of those.

In [1]:
# import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.options.display.max_columns = 1000

In [2]:
# read in the cleaned data
west_nile = pd.read_csv('../data/clean_data')
west_nile.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,WnvPresent,NumMosquitos,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,CodeSum
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,0,2,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ
2,2007-05-29,"1100 South Peoria Street, Chicago, IL 60608, USA",CULEX RESTUANS,11,S PEORIA ST,T091,"1100 S PEORIA ST, Chicago, IL",41.862292,-87.648860,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ
3,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",CULEX RESTUANS,11,W CHICAGO,T049,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ
4,2007-05-29,"1500 North Long Avenue, Chicago, IL 60651, USA",CULEX RESTUANS,15,N LONG AVE,T153,"1500 N LONG AVE, Chicago, IL",41.907645,-87.760886,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ


In [3]:
def preprocess_data(data):
    '''Function to generate the variables needed for analysis'''
    
    # create dummy variables for WNV-carrying species
    data['pipiens'] = np.where(data['Species'].str.contains('PIPIENS'), 1, 0)
    data['restuans'] = np.where(data['Species'].str.contains('RESTUANS'), 1, 0)
    
    # create dummy variables for Code Summary (weather codes)
    codes = ['TSRA', 'RA', 'BR', 'HZ', 'VCTS', 'DZ', 'TS', 'FG']
    for code in codes:
        data[code] = np.where(data['CodeSum'].str.contains(code), 1, 0)
    
    return data

In [4]:
west_nile = preprocess_data(west_nile)
west_nile.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,WnvPresent,NumMosquitos,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,CodeSum,pipiens,restuans,TSRA,RA,BR,HZ,VCTS,DZ,TS,FG
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ,1,1,0,0,1,1,0,0,0,0
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,0,2,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ,0,1,0,0,1,1,0,0,0,0
2,2007-05-29,"1100 South Peoria Street, Chicago, IL 60608, USA",CULEX RESTUANS,11,S PEORIA ST,T091,"1100 S PEORIA ST, Chicago, IL",41.862292,-87.648860,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ,0,1,0,0,1,1,0,0,0,0
3,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",CULEX RESTUANS,11,W CHICAGO,T049,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ,0,1,0,0,1,1,0,0,0,0
4,2007-05-29,"1500 North Long Avenue, Chicago, IL 60651, USA",CULEX RESTUANS,15,N LONG AVE,T153,"1500 N LONG AVE, Chicago, IL",41.907645,-87.760886,8,0,1,1.5,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95,BR HZBR HZ,0,1,0,0,1,1,0,0,0,0


In [5]:
west_nile.to_csv('../data/preprocessed_data', index = False)

# Modeling with the full dataset

Early attempts at building simple models. Starting point for further analysis.

In [6]:
# import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

# Set up X and y
features = ['pipiens', 'restuans', 'Latitude', 'Longitude', 'NumMosquitos', 'Tavg', 'PrecipTotal', 'StnPressure',
           'BR', 'HZ', 'RA', 'TSRA', 'VCTS', 'DZ', 'TS', 'FG']

X = west_nile[features]
y = west_nile['WnvPresent']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [7]:
dt = DecisionTreeClassifier() # Instantiate model
tree = dt.fit(X_train, y_train) # Fit model
predictions = tree.predict(X_test) # Predict y
tree.score(X_test, y_test) # Score model

0.903855085926614

In [8]:
# Generate confusion matrix
cm = confusion_matrix(y_test, predictions) 

cm_df = pd.DataFrame(cm, columns = ['Predicted No West Nile', 'Predicted West Nile'],
                    index = ['Actual No West Nile', 'Actual West Nile'])

cm_df

,Predicted No West Nile,Predicted West Nile
Actual No West Nile,1924,115
Actual West Nile,92,22


In [9]:
# Standard Scale data
ss = StandardScaler()

X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [10]:
knn = KNeighborsClassifier() # Instantiate model
knn = knn.fit(X_train_scaled, y_train) # Fit model
predictions = knn.predict(X_test_scaled) # Predict y
knn.score(X_test_scaled, y_test) # Score model

0.9447282861124013

In [11]:
# Generate confusion matrix
cm = confusion_matrix(y_test, predictions) 

cm_df = pd.DataFrame(cm, columns = ['Predicted No West Nile', 'Predicted West Nile'],
                    index = ['Actual No West Nile', 'Actual West Nile'])

cm_df

,Predicted No West Nile,Predicted West Nile
Actual No West Nile,2027,12
Actual West Nile,107,7


In [14]:
type(west_nile["Date"][0])

str

In [18]:
west_nile.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'WnvPresent', 'NumMosquitos', 'Station', 'Tmax', 'Tmin', 'Tavg',
       'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'CodeSum', 'pipiens', 'restuans', 'TSRA', 'RA', 'BR', 'HZ',
       'VCTS', 'DZ', 'TS', 'FG'],
      dtype='object')